In [1]:
import matplotlib.pyplot as plt
import numpy as np
import time
from tqdm.auto import tqdm
import pymc as pm

In [2]:
from matplotlib import gridspec
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from matplotlib.patches import Patch
import matplotlib.ticker as mtick
import matplotlib as mpl

mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['axes.titlesize'] = 16
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams["axes.formatter.use_mathtext"] = True
leg_size = 14

def SetGrid(ratio=True):
    fig = plt.figure(figsize=(7, 7))
    if ratio:
        gs = gridspec.GridSpec(2, 1, height_ratios=[3,1]) 
        gs.update(wspace=0.025, hspace=0.1)
    else:
        gs = gridspec.GridSpec(1, 1)
    return fig,gs

In [3]:
keys = ['mass','mult','width','sdms','tau2s','zgs']
labels = ["Jet Mass $m$ [GeV]", "Jet Constituent Multiplicity $M$", "Jet Width $\omega$",
          r"Soft Drop Jet Mass $\rho$", r"N-subjetiness Ratio $\tau_{21}^{\beta=1}$", "Groomed Jet Momentum Fraction $z_g$"]
R_BinVals, resp = {}, {}
T_sim_obs, R_sim_obs, T_data_obs, R_data_obs  = {},{},{},{}

In [4]:
def generate_truth(n_samples: int) -> np.ndarray:
    return np.random.normal(size=n_samples)

def confound(
    samples: np.ndarray,
    resolution: float,
) -> np.ndarray:
    noise = np.random.normal(loc=0., scale=resolution, size=samples.shape)
    return samples + noise

def generate_response(
    n_samples: int, 
    bins: list[float],
    resolution: float,
) -> np.ndarray:
    truth_data = np.random.normal(size=n_samples)
    truth_hist, _ = np.histogram(truth_data, bins=bins)
    observed_data = confound(truth_data, resolution=resolution)
    observed_hist, _ = np.histogram(observed_data, bins=bins)
    migrations, _, _ = np.histogram2d(observed_data, truth_data, bins=bins)
    response = migrations / truth_hist
    np.testing.assert_almost_equal(np.dot(response, truth_hist), observed_hist)
    return truth_hist, response

def generate_pseudoexperiment(
    n_samples: int,
    bins: list[float],
    resolution: float,
) -> dict[str, np.ndarray]:
    truth_data = generate_truth(n_samples)
    truth_hist, _ = np.histogram(truth_data, bins=bins)
    observed_data = confound(truth_data, resolution=resolution)
    observed_hist, _ = np.histogram(observed_data, bins=bins)
    return {"truth": truth_hist, "observed": observed_hist}

In [5]:
def compute_posterior(
    observed_hist: np.ndarray,
    response: np.ndarray,
    lower: np.ndarray,
    upper: np.ndarray,
) -> np.ndarray:
    model = pm.Model()

    with model:
        params = pm.DiscreteUniform(
            "params", 
            lower=lower, 
            upper=upper,
        )
        likelihood = pm.Poisson(
            "likelihood", mu=pm.math.dot(response, params),
            observed=observed_hist,
        )
        trace = pm.sample(draws=50000, tune=10000)
    return trace.posterior.params[0].to_numpy()

def plot_posterior(
    posterior: np.ndarray,
    truth_hist: np.ndarray,
    positions:list[float] = [-4.25, -2.75, -1.5, -0.75, -0.25, 0.25, 0.75, 1.5, 2.75, 4.25],
    xerr: list[float] = [1, 0.75, 0.5, 0.25, 0.25, 0.25, 0.25, 0.5, 0.75, 1],
):
    vp = plt.violinplot(
        posterior,
        positions=positions,
        showextrema=False,
    )
    eb = plt.errorbar(
        x=positions,
        y=truth_hist,
        xerr=xerr,
        fmt=".",
    )
    plt.legend([vp["bodies"][0], eb.lines], ["Posterior", "Truth"])
    plt.show()

In [7]:
for key in keys:
    zjets = np.load("zjets_{}.npz".format(key))
    R_BinVals[key] = zjets["bins"]
    T_sim_obs[key], _ = np.histogram(zjets["t_sim"], bins=R_BinVals[key])
    R_sim_obs[key], _ = np.histogram(zjets["r_sim"], bins=R_BinVals[key])
    T_data_obs[key], _ = np.histogram(zjets["t_data"], bins=R_BinVals[key])
    R_data_obs[key], _ = np.histogram(zjets["r_data"], bins=R_BinVals[key])
    migrations, _, _  = np.histogram2d(zjets["r_sim"], zjets["t_sim"], bins=R_BinVals[key])
    resp[key] = migrations / T_sim_obs[key]

In [8]:
for key, binvals, label in zip(T_sim_obs.keys(), R_BinVals, labels):
    if key != 'width': continue
    t = T_data_obs[key]
    bins = np.array(R_BinVals[key])
    bincents = 0.5*(bins[1:]+bins[:-1])
    posterior = compute_posterior(
        observed_hist = R_data_obs[key],
        response = resp[key],
        lower=t - 5*t**0.5,
        upper=t + 5*t**0.5,
    )
    np.savez_compressed("fpu_data_{}".format(key)+".npz", **{"bins": bincents, "m": R_data_obs[key], "t": t, "fbu": posterior, "R": resp[key]})

    plt.errorbar(x=bincents, y=T_data_obs[key], yerr = np.sqrt(t), ls=":")
    plt.errorbar(x=bincents, y=np.mean(posterior,axis=0), yerr=np.std(posterior,axis=0))
    plt.savefig('fbu_sim_{}.png'.format(key),bbox_inches='tight')
    plt.clf()

ERROR (pytensor.graph.rewriting.basic): Rewrite failure due to: constant_folding
ERROR (pytensor.graph.rewriting.basic): node: Floor([563001.19 ... .81963404])
ERROR (pytensor.graph.rewriting.basic): TRACEBACK:
ERROR (pytensor.graph.rewriting.basic): Traceback (most recent call last):
  File "/global/homes/j/jing/.local/perlmutter/tensorflow2.9.0/lib/python3.9/site-packages/pytensor/graph/rewriting/basic.py", line 1922, in process_node
    replacements = node_rewriter.transform(fgraph, node)
  File "/global/homes/j/jing/.local/perlmutter/tensorflow2.9.0/lib/python3.9/site-packages/pytensor/graph/rewriting/basic.py", line 1081, in transform
    return self.fn(fgraph, node)
  File "/global/homes/j/jing/.local/perlmutter/tensorflow2.9.0/lib/python3.9/site-packages/pytensor/tensor/rewriting/basic.py", line 1110, in constant_folding
    thunk = node.op.make_thunk(node, storage_map, compute_map, no_recycling=[])
  File "/global/homes/j/jing/.local/perlmutter/tensorflow2.9.0/lib/python3.9/sit

Sampling 4 chains for 10_000 tune and 50_000 draw iterations (40_000 + 200_000 draws total) took 21 seconds.


<Figure size 640x480 with 0 Axes>

In [9]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp

tfd = tfp.distributions
tfpl = tfp.layers
tfk = tf.keras
tfkl = tf.keras.layers
tfb = tfp.bijectors

In [10]:
print(tfp.__version__)
print(np.__version__)

0.17.0
1.26.3


In [11]:
def IBU(data, init, r, det_binwidth=1, mc_binwidth=1, it=10):
    
    # initialize the truth distribution to the prior
    phis = [init]
    
    # iterate the procedure
    for i in range(it):
        
        # update the estimate for the matrix m
        m = r * phis[-1]
        m /= (m.sum(axis=1)[:,np.newaxis] + 10**-50)

        # update the estimate for the truth distribution
        # the factors of binwidth show up here to change probabilities into probability densities
        phis.append(np.dot(m.T, data)*det_binwidth/mc_binwidth)
        
    return phis

In [13]:
def MLE(model,ymes,ndim):
    x = tf.Variable(ndim*[1.0/ndim])
    loss = lambda: -model.log_prob(x, bijector_kwargs={'conditional_input': ymes})
    losses = tfp.math.minimize(loss,
                               num_steps=10000,
                               #convergence_criterion=(
                               #     tfp.optimizers.convergence_criteria.LossNotDecreasing(atol=0.001)),
                               trainable_variables=[x],
                               optimizer=tf.optimizers.Adam(learning_rate=0.001))
    return x

def MADE(data_shape, cond_shape):
    # Density estimation with MADE.
    made = tfb.AutoregressiveNetwork(params=2,
                                     hidden_units=[100,100,100], #To be changed when using bigger histograms
                                     event_shape=data_shape,
                                     activation='swish',
                                     conditional=True,
                                     conditional_event_shape=cond_shape,
                                    )
    distribution = tfd.TransformedDistribution(
        distribution=tfd.Sample(tfd.Normal(loc=0., scale=1.), sample_shape=[data_shape]),
        bijector=tfb.MaskedAutoregressiveFlow(made))

    # Construct and fit model.
    x_ = tfkl.Input(shape=(data_shape,), dtype=tf.float32)
    c_ = tfkl.Input(shape=(cond_shape,), dtype=tf.float32)
    log_prob_ = distribution.log_prob(x_, bijector_kwargs={'conditional_input': c_})
    model = tfk.Model([x_,c_], log_prob_)

    model.compile(optimizer=tf.optimizers.Adam(learning_rate=4e-5),loss=lambda _, log_prob: -log_prob)
    return model, distribution

def NPU(ymes,tsim,Rin,N,key):
    # Inputs: 
    # ymes: Measured data provided in a histogram with N bins (N,)
    # tsim: Simulated truth used to make the response
    # Rin: Detector resolution matrix. First coordinate is the measured value and second coordinate is the truth level. (M,M)
    # N: Total number of observations
    # Returns samples from p(true|measured).  Would normally want the mode over true, which is equivalent to the MLE given p(true) is uniform.

    M = 1500000 # number of sim used for the response
    ts = []
    lower = [max(0., tsim[i]-5*tsim[i]**0.5) for i in range(len(tsim))] # need positive values for poisson later
    upper = tsim + 5*tsim**0.5
    for k in range(len(ymes)):
        ts.append( np.random.uniform(lower[k], upper[k], M) ) # M values with len(ymes) bins
    ts = np.array(ts).T
    # print(ts)
    # print(np.sum(ts,-1,keepdims=True)) # sum over all bins, keep dim M (total sim data count)
    
    # ts = np.random.uniform(0,1,(M,len(ymes))) #M values with B bins
    ts=N * ts/np.sum(ts,-1,keepdims=True) # N is total measured data count
    # print(ts)
    print(np.sum(ts,-1))

    #pass the uniform prior thru the input response
    ms = []
    for j in range(len(ts)):
        if j % 100000 == 0: 
            print(f"{j}/{len(ts)}") # len(ts) is M
            # print("len(ts[j]):", len(ts[j])) # len(ts[j]) is number of bins
        # for i in range(len(ts[j])):
        #     print("ts[j][i]:", ts[j][i]) # len(ts[j]) is number of bins
        m_hold = [np.random.poisson(ts[j][i]) for i in range(len(ts[j]))] #stat fluctuations
        m_holds = np.random.multinomial(m_hold[0],Rin[:,0])
        for i in range(1, len(ts[j])):
            m_holds += np.random.multinomial(m_hold[i],Rin[:,i])
        ms += [m_holds]
        pass
    ts = np.array(ts)
    ms = np.array(ms)

    n = len(ts)
    x = ms #conditional feature
    y = ts #learn p(y|x)
    nx = N
    ny = N

    #Normalize the total number of events to make the NF easier to train
    x = x/float(nx)
    y = y/float(ny)

    model,dist = MADE(y.shape[1],x.shape[1])
    # Fit.
    batch_size = 16384
    myhistory = model.fit([y,x],
                          y=np.zeros((len(x),0), dtype=np.float32), #dummy labels
                          batch_size=batch_size,
                          epochs=1200,
                          verbose = 1)

    plt.plot(myhistory.history['loss'][10:-1])
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.savefig("npu_loss_{}".format(key)+".pdf")
    plt.clf()

    # mle = MLE(dist,ymes/float(nx),y.shape[-1])
    # print(mle)
    nsample = 100000
    mle = MLE(dist,ymes/float(nx),y.shape[-1]).numpy()
    output = dist.sample(nsample, bijector_kwargs={'conditional_input': np.tile(ymes/float(nx),nsample).reshape([nsample,len(ymes)])}).numpy()
    return output*ny, mle*ny, dist

In [27]:
import corner

for key, binvals, label in zip(T_sim_obs.keys(), R_BinVals, labels):
    if key == 'mass' or key == 'sdms': continue
    zjets = np.load("zjets_{}.npz".format(key))
    bins = np.array(zjets["bins"])
    nbins = len(bins)-1
    p = np.array([1./nbins for i in range(1, nbins+1)])
    bincents = 0.5*(bins[1:]+bins[:-1])
    binsh = 0.07*(bins[1:]-bins[:-1])
    
    H_pT = zjets["H_pT"]
    H_pT_data = zjets["H_pT_data"]
    H_norm_pT = H_pT / H_pT.sum(axis=1, keepdims=True)
    H_norm_pT_data = H_pT_data / H_pT_data.sum(axis=1, keepdims=True)

    T_mc = np.sum(H_pT,axis=1)
    D_mc = np.sum(H_pT,axis=0)
    T_data = np.sum(H_pT_data,axis=1)
    D_data = np.sum(H_pT_data,axis=0)
    N_t = np.sum(T_data)
    t = T_data

    fbu=np.load("fpu_data_{}".format(key)+".npz")
    posterior = fbu['fbu'] 
    print("posterior:", posterior.shape)
    fbu_mean = np.mean(posterior,axis=0)
    fbu_std = np.std(posterior,axis=0)
    print(posterior.shape)
    ibu = IBU(D_data, T_mc, H_norm_pT.T, 1, 1, 15)[15] 
    
    N = D_data.sum()
    # npu = NPU(D_data, T_mc, H_norm_pT.T, N, key)
    # np.savez_compressed("npu_sim_{}".format(key)+".npz", **{"bins": bincents, "m": D_data, "t": T_data, "ibu": ibu,  
    #                                                     "npu": npu[0], "mle": npu[1] })
    
    npu_saved=np.load("npu_data_{}".format(key)+".npz")
    mean = npu_saved['npu'].mean(axis=0)
    std = npu_saved['npu'].std(axis=0)
    mle = npu_saved['mle'] 

    fig,gs = SetGrid(ratio=True) 
    ax0 = plt.subplot(gs[0])
    plt.xticks(fontsize=0)
    ax1 = plt.subplot(gs[1],sharex=ax0)
    
    ax0.fill_between(bins, np.insert(t, len(t), np.array(t[-1])), step='post', alpha=0.3, color='tab:blue', label='Truth')
    ax0.errorbar(x=bincents-binsh, y=ibu, label='IBU', color='tab:red',
                 marker='o', linestyle='None', ms = 10, elinewidth=3, capsize=3)
    ax0.errorbar(x=bincents+binsh,y=fbu_mean, yerr=fbu_std, label='FBU', color='tab:green', 
                 marker='v', linestyle='None', ms = 10, elinewidth=3, capsize=3)  
    # ax0.errorbar(x=bincents, y=npu[1], yerr=npu[0].std(axis=0), label='NPU', color='tab:orange', 
    #              marker='^', linestyle='None', ms = 10, elinewidth=3, capsize=3)
    ax0.errorbar(x=bincents, y=mle,   yerr=std, label='NPU mle', color='tab:orange', 
                 marker='^', linestyle='None', ms = 10, elinewidth=3, capsize=3)
    # ax0.errorbar(x=bincents, y=mean,   yerr=std, label='NPU mean', color='tab:purple', 
    #              marker='^', linestyle='None', ms = 10, elinewidth=3, capsize=3)
    ax0.set_ylabel("Normalized Cross Section [GeV$^{-1}$]", fontsize=16)
    # ax0.set_title(label, fontsize=18)
    ax0.legend(frameon=False, loc="upper right", fontsize=18)
    ax0.tick_params(axis='y', which='major', labelsize=16)
    ax0.ticklabel_format(scilimits=(-3,3), useMathText = True)
    ax0.set_ylim(0, 1.3 * np.max(t))

    r_fbu = np.divide(np.mean(posterior,axis=0), T_data)
    r_ibu = np.divide(ibu, T_data)
    r_npu = np.divide(mle, T_data)
    # r_npu = np.divide(npu[1], T_data)
    # ax1.grid(True, linestyle='dashed', linewidth=1)
    ax1.axhline(y=1.0, color='tab:blue', linestyle='-', linewidth=2, alpha=0.5)
    ax1.set_ylabel("Ratio to Truth", fontsize=16)
    ax1.set_xlabel(label, fontsize=16)
    ax1.tick_params(axis='both', which='major', labelsize=16)
    ax1.set_ylim(0.5, 1.5)
    ax1.errorbar(x=bincents, y=r_ibu, color='tab:red',   marker='o', 
                 linewidth=3,linestyle="None",label='IBU', ms = 10, elinewidth=3, capsize=3)
    ax1.errorbar(x=bincents, y=r_fbu,  color='tab:green',  marker='v',
                 linewidth=3,linestyle="None", label='FBU', ms = 10, elinewidth=3, capsize=3)
    ax1.errorbar(x=bincents,   y=r_npu, color='tab:orange', marker='^',
                 linewidth=3,linestyle="None", label='NPU mle', ms = 10, elinewidth=3, capsize=3)

    fig.savefig("npu_data_{}".format(key)+".pdf")
    fig.show()
    fig.clf()

    # figure = corner.corner(
    # npu_saved['npu'],
    # truths=T_data,
    # hist_kwargs={"color": 'tab:orange', "alpha": 0.3, "fill": True, },
    # color="tab:orange",
    # smooth=True,
    # plot_contours=True,
    # plot_density=False,
    # plot_datapoints=False,
    # fill_contours=True,
    # # show_titles=True,
    # # title_kwargs={"fontsize": 12},
    # quantiles=[0.16, 0.5, 0.84],
    # max_n_ticks=0,
    # top_ticks=False,
    # )  
    # corner.corner(
    # posterior,#[:, 1:-1],
    # fig=figure,
    # smooth=True,
    # plot_contours=True,
    # plot_density=False,
    # plot_datapoints=False,
    # fill_contours=True,
    # color="tab:green",
    # )
    # corner.overplot_lines(figure, T_data, color="deepskyblue", linestyle='dashed', linewidth=4)
    # corner.overplot_points(figure, T_data[None], color="deepskyblue", marker='o', markersize=14)
    # figure.savefig('corner_both_{}'.format(key)+'.pdf')
    # figure.show()
    # figure.clf()

posterior: (50000, 3)
(50000, 3)
posterior: (50000, 5)
(50000, 5)
posterior: (50000, 6)
(50000, 6)
posterior: (50000, 3)
(50000, 3)


<Figure size 700x700 with 0 Axes>

<Figure size 700x700 with 0 Axes>

<Figure size 700x700 with 0 Axes>

<Figure size 700x700 with 0 Axes>

In [26]:
for key, binvals, label in zip(T_sim_obs.keys(), R_BinVals, labels):
    if key == 'mass' or key == 'sdms': continue
    zjets = np.load("zjets_{}.npz".format(key))
    bins = np.array(zjets["bins"])
    nbins = len(bins)-1
    p = np.array([1./nbins for i in range(1, nbins+1)])
    bincents = 0.5*(bins[1:]+bins[:-1])
    binsh = 0.1*(bins[1:]-bins[:-1])
    
    H_pT = zjets["H_pT"]
    H_pT_data = zjets["H_pT_data"]
    H_norm_pT = H_pT / H_pT.sum(axis=1, keepdims=True)

    T_mc = np.sum(H_pT,axis=1)
    D_mc = np.sum(H_pT,axis=0)
    T_data = np.sum(H_pT_data,axis=1)
    D_data = np.sum(H_pT_data,axis=0)
    N_t = np.sum(T_data)
    t = T_data

    fbu=np.load("fpu_data_{}".format(key)+".npz")
    posterior = fbu['fbu']
    ibu = IBU(D_data, T_mc, H_norm_pT.T, 1, 1, 15)[15] 
    
    N = D_data.sum()
    # npu = NPU(D_data, T_mc, H_norm_pT.T, N, key)
    # np.savez_compressed("npu_{}".format(key)+".npz", **{"bins": bincents, "m": D_data, "t": T_data, "ibu": ibu,
    #                                                     "mean": npu[0].mean(axis=0), "std": npu[0].std(axis=0), "mle": npu[1]})
    
    npu_saved=np.load("npu_data_{}".format(key)+".npz")
    mean = npu_saved['npu'].mean(axis=0)
    std = npu_saved['npu'].std(axis=0)
    mle = npu_saved['mle'] 

    fig,gs = SetGrid(ratio=True) 
    ax0 = plt.subplot(gs[0])
    plt.xticks(fontsize=0)
    ax1 = plt.subplot(gs[1],sharex=ax0)

    # Log-y plots
    ax0.fill_between(bins, np.insert(t, len(t), np.array(t[-1])), step='post', alpha=0.3, color='tab:blue', label='Truth')
    ax0.errorbar(x=bincents-binsh, y=ibu, label='IBU', color='tab:red',
                 marker='o', linestyle='None', ms = 10, elinewidth=3, capsize=3)
    ax0.errorbar(x=bincents, y=mle,   yerr=std, label='NPU', color='tab:orange', 
                 marker='^', linestyle='None', ms = 10, elinewidth=3, capsize=3)
    # ax0.errorbar(x=bincents,       y=npu[1],      yerr=npu[0].std(axis=0), label='NPU', color='tab:orange', marker='X', linestyle='None')
    ax0.errorbar(x=bincents+binsh,y=np.mean(posterior,axis=0), yerr=np.std(posterior,axis=0), label='FBU', color='tab:green', 
                 marker='v', linestyle='None', ms = 10, elinewidth=3, capsize=3)   
    ax0.set_ylabel("Normalized Cross Section [GeV$^{-1}$]", fontsize=16)
    ax0.legend(frameon=False, loc="upper right", fontsize=18)
    ax0.tick_params(axis='y', which='major', labelsize=16)
    ax0.set_yscale('log')
    ax0.set_ylim(0.1*np.min(t), 10 * np.max(t))

    r_fbu = np.divide(np.mean(posterior,axis=0), T_data)
    r_ibu = np.divide(ibu, T_data)
    r_npu = np.divide(mle, T_data)
    ax1.axhline(y=1.0, color='tab:blue', linestyle='-', linewidth=2, alpha=0.5)
    ax1.set_ylabel("Ratio to Truth", fontsize=16)
    ax1.set_xlabel(label, fontsize=16)
    ax1.tick_params(axis='both', which='major', labelsize=16)
    ax1.set_ylim(0.5, 1.5)
    # ax1.grid(True, linestyle='dashed', linewidth=1)
    ax1.errorbar(x=bincents, y=r_ibu, color='tab:red',   marker='o', 
                 linewidth=3,linestyle="None",label='IBU', ms = 10, elinewidth=3, capsize=3)
    ax1.errorbar(x=bincents, y=r_fbu,  color='tab:green',  marker='v',
                 linewidth=3,linestyle="None", label='FBU', ms = 10, elinewidth=3, capsize=3)
    ax1.errorbar(x=bincents,   y=r_npu, color='tab:orange', marker='^',
                 linewidth=3,linestyle="None", label='NPU', ms = 10, elinewidth=3, capsize=3)
    fig.savefig("npu_{}_logy".format(key)+".pdf")
    fig.show()
    fig.clf()

<Figure size 700x700 with 0 Axes>

<Figure size 700x700 with 0 Axes>

<Figure size 700x700 with 0 Axes>

<Figure size 700x700 with 0 Axes>